# Data Wrangling Using OpenStreetMap
Max Sydow, WGU BS Data Management and Data Analytics, Udacity Data Wrangling course


# Introduction

     The central portion of the city of Minneapolis, Mn was extracted from the OpenStreetMap Overpass API as an XML file.  I chose this city, because I grew up there and am currently missing it's mild summers.  Data such as street names, and zip codes can be extracted from tags in the XML file using Python's iterparse() method.  The data as elements inside the tags were then used to populate csv's.  These csv's were then used to populate tables in a SQL data base using Python's sqlite3 package.  From here the data can be explored using queries, and further cleaned with Python.

# Preparing the Data

    The size of the Minneapolis.osm file is 66.9 MB.  The following code loads the file, iterativly parses it, then writes to csv's.   Their sizes are as follows:
    nodes.csv             23.7  MB   
    nodes_tags.csv        2.98 MB
    ways.csv              2.94  MB
    ways_nodes.csv        9.95 MB
    ways_tags.csv         4.85 MB

In [20]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
from unittest import TestCase

import cerberus

import schema

OSM_PATH = "Minneapolis.osm"

NODES_PATH = "nodes1.csv"
NODE_TAGS_PATH = "nodes_tags1.csv"
WAYS_PATH = "ways1.csv"
WAY_NODES_PATH = "ways_nodes1.csv"
WAY_TAGS_PATH = "ways_tags1.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

         

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    
    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
                
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['id'] = element.attrib['id']
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['id'] = element.attrib['id']
                    way_tag['key'] = child.attrib['k']
                    way_tag['type'] = 'addr'
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
               
                
    return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()     

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, str) else v) for k, v in row.items()  
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                    
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    process_map(OSM_PATH, validate=True)


The number of each tag can be found using this script.  It'a worth pointing out that the 'nodes' tags include address information about places of interest, and the 'ways' tags describe streets.  These particular tags will be used in later exploration and cleaning.


In [4]:
# count number of each tag

import xml.etree.cElementTree as ET
import pprint

def count_tags(filename):
        # YOUR CODE HERE
        tag_count = {}
        for _, element in ET.iterparse(filename, events=("start",)):
            add_tag(element.tag, tag_count)
        return tag_count
        
def add_tag(tag, tag_count):
    if tag in tag_count:
        tag_count[tag] += 1
    else:
        tag_count[tag] = 1


def test():

    tags = count_tags('Minneapolis.osm')
    pprint.pprint(tags)

    

if __name__ == "__main__":
    test()

{'bounds': 1,
 'member': 49664,
 'meta': 1,
 'nd': 335597,
 'node': 269731,
 'note': 1,
 'osm': 1,
 'relation': 739,
 'tag': 200062,
 'way': 45657}


One issue I noticed while first trying to create the SQL tables is that the elements in the csv files contained byte data that sqlite3 didn't seem to like.  An sample of the nodes1 csv is shown using a data frame.   The function below this output cleans the elements.

In [5]:
import pandas as pd
nodes_df = pd.read_csv('nodes1.csv')
nodes_df.head(1)

,b'id',b'lat',b'lon',b'user',b'uid',b'version',b'changeset',b'timestamp'
0,b'33295121',b'44.9748235',b'-93.2610554',b'lizzz_msp',b'992965',b'7',b'32418946',b'2015-07-05T06:34:35Z'
1,b'33295122',b'44.9748579',b'-93.2611898',b'Mulad',b'502142',b'5',b'9830179',b'2011-11-14T23:04:24Z'
2,b'33295123',b'44.9748942',b'-93.2612886',b'jumbanho',b'38487',b'4',b'7161078',b'2011-02-02T01:33:47Z'
3,b'33295124',b'44.9753793',b'-93.2624228',b'lizzz_msp',b'992965',b'6',b'32418946',b'2015-07-05T06:34:35Z'
4,b'33295319',b'44.9660432',b'-93.3231054',b'Mulad',b'502142',b'4',b'25885566',b'2014-10-05T21:25:25Z'


In [21]:
# cleaning format of elements in csv files - getting rid of b' at beginning and ' at end 

def clean_csv(infile, outfile):
    inputfile = open(infile, 'r')
    outputfile = open(outfile, 'w')
    for row in inputfile:
        new_row = row.replace("b'", '')
        new_row_2 = new_row.replace("'", '')
        #print(new_row_2)
        outputfile.write(new_row_2+'\n')
    return outputfile

clean_csv("nodes1.csv", "nodes.csv")    
clean_csv("nodes_tags1.csv", "nodes_tags.csv")
clean_csv("ways1.csv", "ways.csv")
clean_csv("ways_nodes1.csv", "ways_nodes.csv")
clean_csv("ways_tags1.csv", "ways_tags.csv")

<_io.TextIOWrapper name='ways_tags.csv' mode='w' encoding='cp1252'>

# Creating the Database

Now the database and tables can be created.  The tables include: Nodes, and Ways which each have a primary key; the Nodes_Tags, Ways_Tags, and Ways_Nodes tables reference the first two tables via foreign keys.  Some queries are run to explore basic aspects.

In [7]:
'''  Creating tables using sqlite, database name is Minneapolis.db
     Note the following key relationships:
        Nodes has id as primary key
        Ways has id as primary key
        Nodes_Tags has id as foreign key referencing the Nodes table primary key
        Ways_Tags has id as foreign key referencing the Ways table primary key
        Ways_Nodes has id as foreign key referencing the Ways table primary key
            and node_id as foreign key referencing the Nodes table primary key
'''

import sqlite3
import pandas as pd
from pandas import DataFrame
dbm = db = sqlite3.connect(':memory:') # data base stored in local memory
sqlite3.connect('Minneapolis.db')
c = db.cursor()

# Create Nodes table
c.execute("CREATE TABLE Nodes ([id] integer primary key not null, [lat] float, [lon] float, [user] text, [uid] integer, [version] text, [changeset] integer, [timestamp] text);")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\nodes.csv')
read_clients.to_sql('Nodes', db, if_exists='append', index = False)
db.commit()

# Create Nodes_Tags table
c.execute("CREATE TABLE Nodes_Tags ([id] integer, [key] text, [value] text, [type] text, foreign key (id) references nodes(id));")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\nodes_tags.csv')
read_clients.to_sql('Nodes_Tags', db, if_exists='append', index = False)
db.commit()

# Create Ways table
c.execute("CREATE TABLE Ways ([id] integer primary key not null, [user] text, [uid] integer, [version] text, [changeset] integer, [timestamp] text);")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\ways.csv')
read_clients.to_sql('Ways', db, if_exists='append', index = False)
db.commit()

# Create Ways_Tags table
c.execute("CREATE TABLE Ways_Tags ([id] integer, [key] text, [value] text, [type] text, foreign key (id) references ways(id));")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\ways_tags.csv')
read_clients.to_sql('Ways_Tags', db, if_exists='append', index = False)
db.commit()

# Create Ways_Nodes table
c.execute("CREATE TABLE Ways_Nodes ([id] integer, [node_id] integer, [position] integer, foreign key (id) references ways(id), foreign key (node_id) references nodes(id));")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\ways_nodes.csv')
read_clients.to_sql('Ways_Nodes', db, if_exists='append', index = False)
db.commit()

The number of nodes, ways, unique users can be found using the following queries.  

In [17]:
c.execute('''select count(*) from Nodes''')
print(c.fetchall())

[(1021629,)]


In [15]:
c.execute('''select count(*) from Ways''')
print(c.fetchall())

[(45657,)]


In [27]:
c.execute('''select count(distinct(u.uid))
             from (select uid from nodes union all select uid from ways) u;''')
print(c.fetchall())

[(993,)]


The top 10 contributing users can be found.

In [28]:
c.execute('''select u.user, count(*) as num
            from (select user from Nodes union all select user from Ways) u
            group by u.user
            order by num desc
            limit 10;''')
print(c.fetchall())

[('iandees', 255732), ('Mulad', 239881), ('stucki1', 139264), ('DavidF', 42753), ('woodpeck_fixbot', 25837), ('sota767', 25510), ('neuhausr', 24357), ('rhardy', 23922), ('houston_mapper1', 23213), ('Arun Balaji', 17339)]


The number of users with only one contribution can also be queried.

In [29]:
c.execute('''select count(*) from
             (select u.user, count(*) as num
              from (select user from nodes union all select user from ways) u
              group by u.user
              having num = 1) use;''')
print(c.fetchall())

[(174,)]


# Cleaning Street Names and Zip Codes

Street names, and the number of their occurances can be queried.  

In [9]:
c.execute('''select tags.value, count(*) as count
             from (select * from Nodes_Tags union all select * from Ways_Tags) tags
             where tags.key = 'street'
             group by tags.value
             order by count desc;''')
print(c.fetchall())

[('Lyndale Avenue South', 129), ('West Lake Street', 120), ('Nicollet Avenue South', 119), ('Hennepin Avenue South', 95), ('Southeast University Avenue', 79), ('Southeast 4th Street', 72), ('Southeast Washington Avenue', 62), ('Southeast 7th Street', 52), ('Bryant Avenue South', 47), ('Southeast 5th Street', 44), ('5th Street SE', 39), ('Southeast 8th Street', 39), ('University Avenue SE', 39), ('Washington Avenue South', 39), ('Lakeview Avenue South', 38), ('Southeast 3rd Avenue', 35), ('Southeast 8th Avenue', 35), ('8th St SE', 34), ('East Lake Street', 32), ('Holmes Avenue South', 32), ('Southeast 14th Avenue', 32), ('Humboldt Avenue South', 31), ('East 38th Street', 28), ('East Franklin Avenue', 28), ('1st Avenue South', 27), ('Colfax Avenue South', 26), ('Northeast Buchanan Street', 23), ('Boardwalk Ave', 22), ('University Avenue Southeast', 21), ('19th Avenue South', 20), ('4th Street SE', 20), ('Cedar Avenue South', 20), ('Northeast Pierce Street', 20), ('Southeast Delaware Stre

Inconsistencies with street names and abbreviations such as 'Ave', 'Av', and 'Av.' all being used to describe 'Avenue' can be cleaned up with Python.  The mapping in the code below converts most of the inconsistencies that I found.  

In [10]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "Minneapolis.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) # find street names



expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Av": "Avenue",
            "Ave": "Avenue",
            "Rd.": "Road",
            "Pkwy": "Parkway",
            "Rd/Pkwy": "Parkway",
            "Blvd": "Boulevard",
            "Pl": "Place",
            "Terr": "Terrace",
            "Ln": "Lane",
            "N": "North",
            "N.": "North",
            "S": "South",
            "S.": "South",
            "E": "East",
            "E.": "East",
            "W": "West",
            "W.": "West",
            "NE": "Northeast",
            "SE": "Southeast",
            "Norteast": "Northeast"            
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)



def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")



def audit(osmfile):
    osm_file = open(osmfile, "r", encoding ="utf8")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start","end")):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()                
    
    return street_types


def update_name(name, mapping):
    # match street abbreviations according to the above mapping, and replace with words
    m = street_type_re.search(name)
    better_name = name    
    if m and  m.group() in mapping:
        better_name = name.replace(m.group(), mapping[m.group()])
    return better_name



def test():
    st_types = audit(OSMFILE)
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print(name, "=>", better_name)
                              

if __name__ == '__main__':
    test()

{'280': {'E Hennepin Ave, #280'},
 'Av': {'Lagoon Av', 'Hennepin Av'},
 'Ave': {'Boardwalk Ave',
         'E Hennepin Ave',
         'N Washington Ave',
         'S Washington Ave'},
 'Blvd': {'Stinson Blvd', 'Bottineau Blvd'},
 'Bridge': {'Washington Avenue Bridge'},
 'Curve': {'Hall Curve'},
 'E': {'Hennepin Ave E'},
 'East': {'18th Avenue North East'},
 'Ln': {'Harry Davis Ln'},
 'Mall': {'Nicollet Mall'},
 'N': {'14th Ave N',
       '3rd St N',
       '6th Ave N',
       '913 Plymouth Ave N',
       'Glenwood Ave N',
       'Plymouth Ave N'},
 'N.': {'Washington Ave. N.'},
 'NE': {'1st Avenue NE',
        'Broadway St NE',
        'Central Ave NE',
        'Fillmore St. NE',
        'Harrison St NE',
        'Johnson Street NE',
        'Marshall Street NE'},
 'Norteast': {'30th Avenue Norteast'},
 'North': {'13th Street North',
           '1st Avenue North',
           '1st Street North',
           '2nd Avenue North',
           '2nd Street North',
           '3rd Avenue North',


Some innacuracies in zip codes were also found.  This query returns counts of zip codes.  I was a little surprised to not find many inconsistencies here.  There is one zip code that only has 3 digits.  While I think it's fine to include 4 digit extensions with zip codes, it feels more consistent to have them all appear in the 5-digit format.  

In [11]:
c.execute('''select tags.value, count(*) as count
             from (select * from Nodes_Tags union all select * from Ways_Tags) tags
             where tags.key = 'postcode'
             group by tags.value
             order by count desc;''')
print(c.fetchall())

[('55414', 671), ('55408', 390), ('55455', 209), ('55404', 140), ('55406', 113), ('55405', 90), ('55401', 75), ('55454', 62), ('55403', 60), ('55416', 49), ('55413', 47), ('55407', 46), ('55418', 42), ('55415', 38), ('55402', 36), ('55409', 19), ('55411', 12), ('55455-0153', 5), ('55422', 3), ('55455-0367', 3), ('55412', 2), ('55414-3026', 2), ('55414-3063', 2), ('55455-3002', 2), ('55101', 1), ('554', 1), ('55402-2368', 1), ('55402-9998', 1), ('55403-2006', 1), ('55403-3933', 1), ('55411-4398', 1), ('55414-2015', 1), ('55414-2016', 1), ('55414-2051', 1), ('55414-2475', 1), ('55414-2919', 1), ('55414-2921', 1), ('55414-3017', 1), ('55414-3022', 1), ('55414-3029', 1), ('55415-1226', 1), ('55418-3510', 1), ('55418-4714', 1), ('55419', 1), ('55423', 1), ('55454-1015', 1), ('55454-1313', 1), ('55455-0200', 1), ('55455-0226', 1), ('55455-0236', 1), ('55455-0237', 1), ('55455-0240', 1), ('55455-0366', 1), ('55455-0425', 1), ('55455-0441', 1), ('55455-0442', 1), ('55455-0485', 1), ('55455-050

This python code cleans zip codes in a similiar manner to the address auditing script.  The expected zip codes were found from https://www.zip-codes.com/city/mn-minneapolis.asp.  

In [14]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "Minneapolis.osm"

expected = ['5540\d{1}', '5541\d{1}', '5542\d{1}', '5543\d{1}', '5544\d{1}', '55450', '55454', '55455', '55458',
            '55459', '55460', '55467', '55470', '55472', '55474', '55478', '55479', '55480', '55483', '55484'
            '55485', '55486', '55487', '55488']
zip_map ={}
postal_re = re.compile(r'(\d{5})')  # find 5 digit postal code


def audit_post_code(zip_codes, post_code):
    m = postal_re.search(post_code)
    if m:
        zip_code = m.group()
        if zip_code not in expected:
            zip_codes[zip_code].add(post_code)

def is_post_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit(osmfile):
    osm_file = open(osmfile, "r", encoding ="utf8")
    post_codes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start","end")):            
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_post_code(tag):
                    audit_post_code(post_codes, tag.attrib['v']) 
    osm_file.close()
    return post_codes

print(audit(OSMFILE))

def update_zip(post, zip_map):
    m = postal_re.search(post)
    better_post = post
    if m and m.group()  in zip_map:
        better_post = post.replace(m.group(), mapping[m.group()])
        
    return better_post

def test():
    zip_codes = audit(OSMFILE)

    for zp_code, nodes in zip_codes.items():
        for post in nodes:
            #print(post)
            if len(post) == 5:
                 better_post = update_zip(post, zip_map)
                 print(post, "=>", better_post)
                              

if __name__ == '__main__':
    test()

defaultdict(<class 'set'>, {'55414': {'55414-3063', '55414-2015', '55414-3022', '55414-2475', '55414-2921', '55414', '55414-3026', '55414-2016', '55414-2051', '55414-3029', '55414-3017', '55414-2919'}, '55406': {'55406'}, '55416': {'55416'}, '55408': {'55408'}, '55403': {'55403', '55403-2006', '55403-3933'}, '55402': {'55402', '55402-2368', '55402-9998'}, '55405': {'55405'}, '55404': {'55404'}, '55401': {'55401'}, '55407': {'55407'}, '55409': {'55409'}, '55411': {'55411-4398', '55411'}, '55418': {'55418-4714', '55418-3510', '55418'}, '55415': {'55415-1226', '55415'}, '55413': {'55413'}, '55422': {'55422'}, '55412': {'55412'}, '55101': {'55101'}, '55423': {'55423'}, '55419': {'55419'}})
55414 => 55414
55406 => 55406
55416 => 55416
55408 => 55408
55403 => 55403
55402 => 55402
55405 => 55405
55404 => 55404
55401 => 55401
55407 => 55407
55409 => 55409
55411 => 55411
55418 => 55418
55415 => 55415
55413 => 55413
55422 => 55422
55412 => 55412
55101 => 55101
55423 => 55423
55419 => 55419


# Further Exploration

Querying phone numbers reveals inconsistencies in thier formats.  Using Python to clean that data similiar to what was done with street names and zip codes is another task that could be performed.  

In [30]:
c.execute('''select tags.value, count(*) as count
             from (select * from Nodes_Tags union all select * from Ways_Tags) tags
             where tags.key = 'phone'
             group by tags.value
             order by count desc;''')
print(c.fetchall())

[('612-273-8383', 3), ('+1-763-221-5525', 2), ('16123318100', 2), ('612-331-5000', 2), ('612-331-7474', 2), ('612-370-4925', 2), ('612-822-6088', 2), ('612-823-8095', 2), ('612-870-1976', 2), ('800-275-8777', 2), ('(414) 839-5565', 1), ('(612) 227-9635', 1), ('(612) 236-4429', 1), ('(612) 238-8888', 1), ('(612) 248-8111', 1), ('(612) 259-7519', 1), ('(612) 331-4724', 1), ('(612) 332-4393', 1), ('(612) 333-4242', 1), ('(612) 338-8401', 1), ('(612) 341-2112', 1), ('(612) 379-2259', 1), ('(612) 404 0056', 1), ('(612) 465-8780', 1), ('(612) 481-7551', 1), ('(612) 607-5016', 1), ('(612) 643-1933', 1), ('(612) 781-6529', 1), ('(612) 789-7238', 1), ('(612) 789-8851', 1), ('(612) 823-0977', 1), ('(612) 823-6233', 1), ('(612) 824-6665', 1), ('(612) 825-9922', 1), ('(612) 926-8626', 1), ('(612) 999-4200', 1), ('+ 612 789 4996', 1), ('+1 (612) 371 3100', 1), ('+1 (612) 379-1723', 1), ('+1 (612) 758-0105', 1), ('+1 (612)-354-2453', 1), ('+1 612 204 5000', 1), ('+1 612 206 3920', 1), ('+1 612 208 1

Finally, I think it's interesting to find the most frequent appearing types of places included in the osm file.  

In [32]:
c.execute('''select value, count(*) as num
             from nodes_tags
             where key = 'amenity'
             group by value
             order by num desc
             limit 10;''')
print(c.fetchall())

[('restaurant', 291), ('bicycle_parking', 203), ('bench', 136), ('cafe', 100), ('bicycle_rental', 97), ('fast_food', 57), ('waste_basket', 44), ('parking_entrance', 43), ('pu', 41), ('bar', 38)]
